In [1]:
%cd drive/MyDrive/Colab\ Notebooks/NSI

/content/drive/MyDrive/Colab Notebooks/NSI


In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import math, copy, time
from typing import NoReturn, Optional, Callable, Generator, Iterable, List
import matplotlib.pyplot as plt
import seaborn
from collections import OrderedDict
import pickle
import gensim
import sklearn

seaborn.set_context(context='talk')
%matplotlib inline

In [3]:
with open('data/full_training_set.pkl', 'rb') as f:
  df = pickle.load(f)

In [21]:
# new B1 case
import pandas as pd
ids = list(pd.read_csv('data/신규B1_ids.csv')['idx'])
df = df.iloc[ids]

In [4]:
# new B2 case
import pandas as pd
ids = list(pd.read_csv('data/신규B2_ids.csv')['IDX'])
df = df.iloc[ids]

In [5]:
new_wv = gensim.models.KeyedVectors.load('data/new_w2v.wv')

In [6]:
def vectorize_sent(sent: List[str]) -> np.ndarray:
  return np.array([new_wv[token] for token in sent if token in new_wv])

In [ ]:
#embedded = np.array(df['POSTAG'].apply(vectorize_sent))
df['EMBEDDED'] = df['POSTAG'].apply(vectorize_sent)

In [ ]:
df.head()

In [ ]:
#for cleaning momory
new_wv = 0

In [ ]:
df = df[df['EMBEDDED'].apply(len) > 0].sample(frac=1, random_state=0)

In [ ]:
df.head()

,CONTENT,ARTICLE,BUSINESS,CONSUMER,DOMESTIC,POSTAG,EMBEDDED
81503,여기에 현재 상황을 보여주는 동행지수 순환변동치 역시 99.6에서 100.1로 상승했다.,0,1,0,1,"[여기/Noun, 에/Josa, 현재/Noun, 상황/Noun, 을/Josa, 보여...","[[4.884522, -2.5102544, -0.98372334, -1.691520..."
332396,공립 유치원 종일반 기준 도 수업료가 138만원에서 145만원으로 4.9% 뛰어 ...,0,0,2,1,"[공립/Noun, 유치원/Noun, 종/Noun, 일반/Noun, 기준/Noun, ...","[[4.456582, -0.782104, -4.5609674, 4.3970222, ..."
157251,다음은 전기&#183;전자(-1.13%) 화학(-1.09%) 기계(-0.83%) ...,0,0,0,1,"[다음/Noun, 은/Josa, 전기/Noun, &#/Punctuation, 183...","[[3.8228464, -10.924315, 3.814275, -5.2757435,..."
322365,◇ 런던 금융가 “이건 전쟁이다 ” 영국 금융산업의 심장인 더 시티에서는 뉴욕증권...,0,2,0,2,"[◇/Foreign, 런던/Noun, 금융가/Noun, “/Foreign, 이/De...","[[-8.214905, -0.7775568, -2.3306205, 2.4271498..."
206666,12일 코스닥지수가 크게 오르면서 거래대금은 사상 최대치를 기록했다.,0,1,0,1,"[12일/Number, 코스닥/Noun, 지수/Noun, 가/Josa, 크게/Nou...","[[-4.4134684, 8.968531, 1.5570612, -0.18283427..."


In [ ]:
embedded = np.array(df['EMBEDDED'])
labs_business = np.array(df['BUSINESS'])

In [ ]:
len(embedded), len(labs_business)

(288211, 288211)

In [ ]:
embedded_domestic = np.array(df['EMBEDDED'][df['DOMESTIC'] > 0])
labs_domestic = np.array(df['DOMESTIC'][df['DOMESTIC'] > 0])

In [ ]:
len(embedded_domestic), len(labs_domestic)

(203164, 203164)

In [ ]:
#for cleaning momory
df = 0

In [ ]:
#with open('data/embedded_full_training_set.pkl', 'wb') as f:
#  pickle.dump(embedded, f)


In [ ]:
i = 0
row_size = 64000
total_size = len(embedded)
while (i + 1) * row_size < total_size:
  with open(f'data/embedded_new_b2_business_{i}.pkl', 'wb') as f:
    pickle.dump(embedded[i * row_size:(i + 1) * row_size], f)
  i += 1
print(i, total_size - i * row_size)
with open(f'data/embedded_new_b2_business_{i}.pkl', 'wb') as f:
    pickle.dump(embedded[i * row_size:], f)

4 32211


In [ ]:
with open('data/cleaned_new_b2_business_labels.pkl', 'wb') as f:
  pickle.dump(labs_business, f)

In [ ]:
i = 0
row_size = 64000
total_size = len(embedded_domestic)
while (i + 1) * row_size < total_size:
  with open(f'data/embedded_new_b2_domestic_{i}.pkl', 'wb') as f:
    pickle.dump(embedded_domestic[i * row_size:(i + 1) * row_size], f)
  i += 1
print(i, total_size - i * row_size)
with open(f'data/embedded_new_b2_domestic_{i}.pkl', 'wb') as f:
    pickle.dump(embedded_domestic[i * row_size:], f)

3 11164


In [ ]:
labs_domestic

array([1, 1, 1, ..., 1, 3, 2])

In [ ]:
labs_domestic -= 1
labs_domestic

array([0, 0, 0, ..., 0, 2, 1])

In [ ]:
with open('data/cleaned_new_b2_domestic_labels.pkl', 'wb') as f:
  pickle.dump(labs_domestic, f)